# Python para Lingüistas

Notebook 9: Chunking y Parsing

Alejandro Ariza

Universitat de Barcelona 2022

En este notebook, veremos como hacer procesamiento sintáctico usando NLTK.

En concreto, veremos 2 formas diferentes - chunking y análisis sintáctico completo.

In [1]:
# Importar nltk
import nltk

In [2]:
# Descargar los paquetes importantes para hoy (si no los tenéis ya descargados)
nltk.download("conll2000")

[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\Venelin\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


True

In [3]:
# Importar corpus para chunking
from nltk.corpus import conll2000



(Por qué) Estructura del texto:

- “El significado de una expresión compleja está determinado por el significado de sus expresiones constituyentes y **las reglas usadas para combinarlas**”
    - Estructura formal, no semántica
- “A dog bites a man” vs “A man bites a dog” – en un bag-of-words, ambos textos reciben la misma representación
- Elementos composicionales y no composicionales: “A dog bites John Smith”

Análisis sintáctico y generación:
- “A Dog bites a man” -> quién hizo a quién el qué (¿está bien construida la frase incluso?)
- “Dog”, “bite”, “man” -> ¿Qué tipo de frases pueden ser construidas con estas palabras?

Shallow parsing (chunking):
- Estructura lineal I-O-B. 
- Sin jerarquía. Sin dependencias complejas distantes.
- Identificar constituyentes.
- Más efectivo computacionalmente y con mayor precisión.
- Se puede usar un método clásico (HMM)

In [4]:
# Separa el corpus conll en test y train
test_sents = conll2000.chunked_sents('test.txt')
train_sents = conll2000.chunked_sents('train.txt')

# Mira el formato del corpus anotado
# Imprime el formato original
print("Chunked sentence: {}".format(train_sents[99]))
# Imprime el formato I-O-B
print("Chunked sentence in I-O-B format: {}".format(nltk.chunk.tree2conlltags(train_sents[99])))
# Dibuja un árbol
# El árbol aparecerá en una ventana separada
train_sents[99].draw()

Chunked sentence: (S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)
Chunked sentence in I-O-B format: [('Over', 'IN', 'B-PP'), ('a', 'DT', 'B-NP'), ('cup', 'NN', 'I-NP'), ('of', 'IN', 'B-PP'), ('coffee', 'NN', 'B-NP'), (',', ',', 'O'), ('Mr.', 'NNP', 'B-NP'), ('Stone', 'NNP', 'I-NP'), ('told', 'VBD', 'B-VP'), ('his', 'PRP$', 'B-NP'), ('story', 'NN', 'I-NP'), ('.', '.', 'O')]


Análisis sintáctico completo:
- Asigna una estructura (jerárquica binaria) para una frase, dada una gramática
- Input: Gramática (pre-definida), frase
- Output: todos los árboles posibles (si existen)

Ambigüedad sintáctica (múltiples posibilidades para la misma frase)
- Desambiguación mediante modelos probabilísticos
- Desambiguación basada en reglas (heurística)
- Uso de recursos externos (tales como diccionarios, bases de datos, y corpus anotados)

Gramática Libre de Contexto:
- conjuntos de reglas que expresan las posibles formas de combinar y ordenar símbolos correspondientes a un lenguaje
- Un léxico de palabras y símbolos


- nodos terminales – el léxico del lenguaje (palabras)
- nodos no terminales – generalización de nodos (clases, tales como POS)
- nodo raíz (S)
- derivación – una secuencia de reglas de expansión (izquierda a derecha)

In [5]:
# Un CFG simple
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

# Frase de test
sent = "Mary saw Bob".split()

# Analiza la frase utilizando la gramática
rd_parser = nltk.RecursiveDescentParser(grammar1)

# Imprime todos los árboles válidos
for tree in rd_parser.parse(sent):
    print(tree)

# Dibuja todos los árboles
for tree in rd_parser.parse(sent):
    tree.draw()

(S (NP Mary) (VP (V saw) (NP Bob)))


In [ ]:
# Observa el funcionamiento de múltiples algoritmos de análisis sintáctico

# Analizador Top-down
# - Comienza por “S”
# - Genera un árbol
# - Mapea el árbol a los nodos terminales

nltk.app.rdparser()

In [ ]:
# Analizador Bottom-up
# - Comienza desde los nodos terminales
# - Los agrupa por frases
# - Intenta construir el árbol hasta encontrar la raíz S.
nltk.app.srparser()

In [ ]:
# Tarea 1
# Usa grammar1 para analizar cada frase del siguiente corpus:

corpus = [['a', 'young', 'woman', 'walks', 'in', 'the', 'park'], 
['two', 'young', 'men', 'smile'], 
['a', 'young', 'woman', 'sees', 'two', 'men'], 
['sees', 'two', 'men', 'a', 'young', 'woman'], 
['a', 'young', 'woman', 'sees', 'two', 'old', 'men', 'in', 'the', 'park', 'with', 'a', 'telescope'], 
['a', 'young', 'woman', 'two', 'old', 'men', 'in', 'the', 'park', 'with', 'a', 'telescope', 'sees'], 
['two', 'angry', 'men', 'chase', 'a', 'woman', 'with', 'a', 'telescope'], 
['a', 'woman', 'I', 'know', 'owns', 'a', 'telescope'], 
['a', 'woman', 'I', 'know', 'a', 'telescope']]

In [ ]:
# Tarea 2
# Expande grammar1 con reglas adicionales, de forma que podáis ver múltiples árboles para cada palabra
# Deberíais obtener el siguiente número de soluciones para las frases del corpus:
# “a young woman walks in the park” <- 1 solución
# “two young men smile” <- 1 solución
# “a young woman sees two men” <- 1 solución
# “sees two men a young woman” <- 0 soluciones
# “a young woman sees two old men in the park with a telescope” <- al menos 3 soluciones
# “a young woman two old men in the park with a telescope sees” <- 0 soluciones
# “two angry men chase a woman with a telescope” <- 2 soluciones
# “a woman I know owns a telescope” <- 1 solución
# “a woman I know a telescope” <- 0 soluciones